# 敵対的トレーニング

# 1. 事前準備

## 必要なライブラリのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
tf.compat.v1.disable_eager_execution() #  Eager Execution を無効にする

# ART 関連
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import KerasClassifier

## 通常モデルをロード

In [2]:
normal_model = keras.models.load_model('../../models/mnist.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## ロバストモデルの生成（誤って読者が実行しないようにコメントアウトしています）

In [3]:
# robust_model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=X_train.shape[1:3]), # フラット化のための層
#     keras.layers.Dense(64, activation='relu'), # レイヤ1。64ノードで活性化関数は ReLU 関数
#     keras.layers.Dense(32, activation='relu'), # レイヤ2。32ノードで活性化関数は　ReLU 関数
#     keras.layers.Dense(10, activation='softmax')]) # レイヤ3。10ノードで活性化関数は Softmax 関数

## 敵対的トレーニング前のロバストモデルの保存（誤って読者が実行しないようにコメントアウトしています）

In [4]:
# robust_model.save('../../models/initial_robust_mnist.h5')

## 筆者が生成した敵対的トレーニング前のロバストモデルのロード

In [3]:
robust_model = keras.models.load_model('../../models/initial_robust_mnist.h5')

## ロバストモデルのコンパイル

In [4]:
robust_model.compile(
    optimizer=SGD(learning_rate=0.01), # 最適化アルゴリズムに確率的勾配降下法を利用
    loss='sparse_categorical_crossentropy', # 損失関数に交差エントロピー誤差を利用
    metrics=['accuracy']) # 評価メトリクスに正解率を利用

## MNIST データの取得とスケール変換

In [5]:
# MNIST データを Keras を使って取得
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

# [0, 1] スケールに変換
X_train, X_test = X_train/255.0, X_test/255.0

# 2. 敵対的トレーニング用クラスの開発と敵対的トレーニング

## 敵対的トレーニング用のクラス

In [6]:
class AdversarialTraining:
    """ Adversarial Training
    Attributes:
        robust_model: ロバストモデル
        attack : ART の敵対的サンプル生成用のクラスのインスタンス
    """

    def __init__(self, robust_model, attack):

        # 引数をインスタンス変数にセット
        self.robust_model = robust_model
        self.attack = attack

    def train(self, mini_batch_size, num_epochs, X_train, Y_train, X_test, Y_test):
        """
        敵対的トレーニングを行う
        Args:
            mini_batch_size (int): ミニバッチサイズ
            num_epochs　(int) : エポック数
            X_train (ndarray): 学習データ
            Y_train (ndarray): 正解データ
            X_test (ndarray): テストデータ
            Y_test (ndarray): テストデータの正解データ
        """

        # エポックごとのミニバッチの数
        num_mini_batches = (X_train.shape[0] - 1) // mini_batch_size + 1
        
        # エポックのループ
        for epoch in range(num_epochs):

            # ミニバッチのループ
            for b in range(num_mini_batches):
                
                # ミニバッチを取得
                start = b * mini_batch_size
                end = start + mini_batch_size

                # ミニバッチから敵対的サンプルを生成
                X_adv = self.attack.generate(X_train[start:end], batch_size=mini_batch_size)
                
                # エポックごとにログの出力とテストデータによる評価を行う
                if b == num_mini_batches - 1:
                    # 敵対的サンプルで学習
                    self.robust_model.fit(
                        X_adv, # 学習データ（敵対的サンプル）
                        Y_train[start:end], # 正解データ
                        batch_size=mini_batch_size, # ミニバッチサイズ
                        epochs=1, # エポック数
                        verbose=2, # ログレベル
                        validation_data=(X_test, Y_test)) # エポック毎に評価で使うデータ。テストデータとその正解データを指定
                
                else:
                    # 敵対的サンプルで学習
                    self.robust_model.fit(
                        X_adv, # 学習データ（敵対的サンプル）
                        Y_train[start:end], # 正解データ
                        batch_size=mini_batch_size, # ミニバッチサイズ
                        epochs=1, # エポック数
                        verbose=0) # ログレベル

## AdversarialTraining クラスのインスタンスを生成

In [7]:
# ART 用の　classifier を生成
art_classifier_robust = KerasClassifier(model=robust_model, clip_values=(0, 1))

# ミニバッチサイズ
mini_batch_size = 50

# PGD Attack 用のクラスのインスタンスを生成
attack_train = ProjectedGradientDescent(art_classifier_robust, eps=0.1, eps_step=0.005, max_iter=40, batch_size=mini_batch_size)

# AdversarialTraining のインスタンスを生成
at = AdversarialTraining(robust_model, attack_train) 

## 敵対的トレーニングの実行

In [13]:
at.train(mini_batch_size=mini_batch_size, num_epochs=20, X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test)

Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 1.4703 - accuracy: 0.3400 - val_loss: 0.6317 - val_accuracy: 0.8052
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.9619 - accuracy: 0.5800 - val_loss: 0.4930 - val_accuracy: 0.8538
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.8337 - accuracy: 0.6800 - val_loss: 0.4544 - val_accuracy: 0.8710
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.7710 - accuracy: 0.6800 - val_loss: 0.4361 - val_accuracy: 0.8730
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.7285 - accuracy: 0.7200 - val_loss: 0.4239 - val_accuracy: 0.8757
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.7366 - accuracy: 0.6800 - val_loss: 0.4171 - val_accuracy: 0.8756
Train on 50 samples, validate on 10000 samples
50/50 - 0s - loss: 0.7223 - accuracy: 0.7000 - val_loss: 0.4126 - val_accuracy: 0.8758
Train on 50 samples, validate on 10000 samples
50/50 - 0s - lo

## ロバストモデルの保存（誤って読者が実行しないようにコメントアウトしています）

In [14]:
# robust_model.save('../../models/robust_mnist.h5')

## 筆者が学習させたロバストモデルのロード（ロードする場合はコメントアウトを解除して実行してください）

In [19]:
# robust_model = keras.models.load_model('../../models/robust_mnist.h5')

# 3. モデルの評価

## 通常モデルでテストデータの敵対的サンプルを生成

In [21]:
# ART 用の　classifier を生成
art_classifier_normal = KerasClassifier(model=normal_model, clip_values=(0, 1))

# PGD Attack 用のクラスのインスタンスを生成
attack_normal = ProjectedGradientDescent(art_classifier_normal, eps=0.1, eps_step=0.005, max_iter=40, batch_size=mini_batch_size)

# テストデータの敵対的サンプルを生成
adv_images_normal = attack_normal.generate(X_test)

## ロバストモデルでテストデータの敵対的サンプルを生成

In [25]:
# ART 用の　classifier を生成
art_classifier_robust = KerasClassifier(model=robust_model, clip_values=(0, 1))

# PGD Attack 用のクラスのインスタンスを生成
attack_robust = ProjectedGradientDescent(art_classifier_robust, eps=0.1, eps_step=0.005, max_iter=40, batch_size=mini_batch_size)

# テストデータの敵対的サンプルを生成
adv_images_robust = attack_robust.generate(X_test)

## 通常モデルの評価

In [23]:
normal_model.evaluate(adv_images_normal, Y_test)

10000/10000 [==============================] - 0s 25us/sample - loss: 5.8544 - accuracy: 0.0656


[5.854394119262695, 0.0656]

## ロバストモデルの評価

In [24]:
robust_model.evaluate(adv_images_robust, Y_test)

10000/10000 [==============================] - 0s 28us/sample - loss: 0.6417 - accuracy: 0.7494


[0.6416599189996719, 0.7494]